In [5]:
import pandas as pd
import numpy as np
import jieba

#数据加载
news = pd.read_csv('D:\\开课吧课程资料\\核心能力提升班商务智能方向004期\\L3\\BI3期核心-L3代码\\L3\\Assignment\\sqlResult.csv',engine='python',encoding='gb18030')
print(news.shape)
print(news.head())


(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...   
3      这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [48]:
news.content[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\n'

In [6]:
news[news.content.isna()].head()
news = news.dropna(subset=['content'])
print(news.shape)

(87054, 7)


In [30]:
#加载停用词
with open('chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]

def split_text(text):
    text = text.replace(' ','').replace('\n','')
    text2 = jieba.cut(text)
    result =' '.join([w for w in text2 if w not in stopwords])
    return result
    

In [31]:
corpus = list(map(split_text,[str(i) for i in news.content]))
corpus

['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息',
 '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台',
 '此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A 电池 缩水 可能 刘作 虎 所说 一加 手机 市面 轻薄 大屏 旗舰 设定 按照 目前 掌握 资料 一加 手机 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 6G 8GBRAM 64GB 128GBROM 双 1600 万 摄像头 备货 量 惊喜 京东 泄露 信息 一加 售价 xx99 应该 2799 2899 2999',
 '这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄',
 '原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 事发 到底 发生 些 南 记者 带 一起 还原 现场 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 些 腼腆 90 青年 刘青 介绍 16 早上 

In [32]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
#计算corpus的TF-IDF
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
#计算词频向量
countvector = countvectorizer.fit_transform(corpus)
#将词频向量转化为tfidf向量
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)


(87054, 884)


In [34]:
#标记是否为自己的新闻
label = list(map(lambda source: 1 if '新华社' in str(source) else 0 ,news.source))
label

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [35]:
from sklearn.model_selection import train_test_split
#数据集切分
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(),label,test_size=0.3)
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
y_predict = model.predict(X_test)
print('准确率：',accuracy_score(y_test,y_predict))
print('精确率：',precision_score(y_test,y_predict))
print('召回率：',recall_score(y_test,y_predict))


准确率： 0.8851705785503695
精确率： 0.9587587498328057
召回率： 0.9120366443294596


In [39]:
#使用模型进行风格预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
#compare_new_index有两列
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
copy_news_index = compare_news_index[(compare_news_index['prediction']==1)&(compare_news_index['labels']==0)].index


In [54]:
print(len(copy_news_index))

2862


In [40]:
#实际为新华社的新闻
xinhuashe_news_index = compare_news_index[(compare_news_index['labels']==1)].index
print('可能为copy的新闻条数：',len(copy_news_index))

可能为copy的新闻条数： 2862


In [41]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())
#使用KMeans进行全量文档聚类
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)
print(k_labels.shape)

(87054,)


In [42]:
#创建id_class ID是1-87000 class是1-25
id_class = {index:class_ for index, class_ in enumerate(k_labels)}
from collections import defaultdict
class_id = defaultdict(set)
for index,class_ in id_class.items():
    #只统计新华社发布的class_id
    if  index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
#查找相似的文章
def find_similar_text(cpindex,top=10):
    #只在新华社发布的文章中查找
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    #从大到小排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

In [45]:
cpindex = 3352
print('是否在新华社',cpindex in xinhuashe_news_index)
print('是否在copy_news',cpindex in copy_news_index)

是否在新华社 False
是否在copy_news True


In [46]:
similar_list = find_similar_text(cpindex)
print(similar_list)

[(3134, array([[0.96849134]])), (63511, array([[0.94643202]])), (29441, array([[0.9428342]])), (3218, array([[0.87621844]])), (29615, array([[0.86936332]])), (29888, array([[0.86215823]])), (64046, array([[0.85278237]])), (29777, array([[0.84875426]])), (63974, array([[0.73415186]])), (63975, array([[0.73415186]]))]


In [47]:
print('怀疑抄袭：\n',news.iloc[cpindex].content)
#找一篇相似的原文
similar2 = similar_list[0][0]
print('相似原文：\n',news.iloc[similar2].content)

怀疑抄袭：
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似原文：
 　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建